In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import os
import numpy as np
from datetime import datetime
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import trange, tqdm
from datetime import datetime,timedelta
import random
from pygeocoder import Geocoder
import folium
import branca.colormap as cmp

In [9]:
df_FC_In  = pd.read_csv('E:\Data\divvy\EDA/Nbh_IN_forecastability.csv' ,index_col=0)
df_FC_Out = pd.read_csv('E:\Data\divvy\EDA/Nbh_OUT_forecastability.csv',index_col=0)

In [12]:
df_station = pd.read_csv('E:\Data\divvy/df_station_1721_FINAL.csv',index_col=0)

In [10]:
df_FC_In

,Nbh_row,Nbh_id,ADI,CV2,Forecastability
1,1,1.0,1.321956,1.504304,Lumpy
2,2,1.1,2.370006,2.884210,Lumpy
3,3,2.0,4.619901,5.291060,Lumpy
4,4,2.1,9.540523,10.321384,Lumpy
5,5,2.3,7.573455,8.203588,Lumpy
...,...,...,...,...,...
165,165,77.0,2.229014,2.750419,Lumpy
166,166,77.1,1.758824,2.289547,Lumpy
167,167,77.2,2.762890,4.574981,Lumpy
168,168,77.3,3.617227,4.434217,Lumpy


In [15]:
df_station = df_station.merge(df_FC_In,on='Nbh_id')

In [16]:
df_station

,station_name,lng,lat,Docks in Service,Nbh_id,com_id,date_begin,date_end,Nbh_row,ADI,CV2,Forecastability
0,2112 W Peterson Ave,-87.683593,41.991181,11.0,2.4,2.0,NaN,NaN,6,5.393218,6.124525,Lumpy
1,Clark St & Elmdale Ave,-87.669762,41.990683,13.0,2.4,2.0,NaN,NaN,6,5.393218,6.124525,Lumpy
2,351,-87.606611,41.803023,15.0,39.1,39.0,NaN,NaN,114,3.326658,3.843092,Lumpy
3,Cottage Grove Ave & 43rd St,-87.606587,41.816495,11.0,39.1,39.0,NaN,NaN,114,3.326658,3.843092,Lumpy
4,Cottage Grove Ave & 47th St,-87.606764,41.809857,13.0,39.1,39.0,NaN,NaN,114,3.326658,3.843092,Lumpy
...,...,...,...,...,...,...,...,...,...,...,...,...
822,Wentworth Ave & Cermak Rd,-87.631695,41.853077,17.0,3.4,3.0,2020-06-03,NaN,11,3.645452,4.719925,Lumpy
823,Wentworth Ave & Cermak Rd (Temp),-87.631474,41.853059,19.0,3.4,3.0,2018-01-01,2020-06-04,11,3.645452,4.719925,Lumpy
824,Wallace St & 35th St,-87.641292,41.830621,13.0,34.2,34.0,NaN,NaN,101,6.902124,7.322397,Lumpy
825,Western Ave & 104th St,-87.662560,41.704878,15.0,72.2,72.0,2020-10-06,NaN,158,inf,NaN,Lumpy


In [19]:
df_station.Forecastability.value_counts()

Lumpy      658
Erratic    169
Name: Forecastability, dtype: int64

In [18]:
CHICAGO_COORDINATES = (41.864073, -87.706819)
# for speed purposes
MAX_RECORDS = 100

# create empty map zoomed in on San Francisco
map = folium.Map(location=CHICAGO_COORDINATES, zoom_start=11)

In [20]:
def colormap(Forecastability):
    return {'Lumpy':'red','Erratic':'green'}[Forecastability]

In [21]:
# add a marker for every record in the filtered data
for idx,row in df_station.iterrows():
    folium.Circle(
        location = (row['lat'],row['lng']),
        radius = 2,
        color = colormap(row['Forecastability'])
        ).add_to(map)
#map.add_child(colormap)
display(map)

### Visualizing Divvy Station deployment

In [45]:
df_station.date_begin = df_station.date_begin.fillna('2016-01-01')

In [46]:
df_station.insert(0,'station_age', df_station.date_begin_filled.apply(lambda x:(datetime.now() - datetime.strptime(x, "%Y-%m-%d")).days))

In [47]:
df_station

,station_age,station_name,lng,lat,Docks in Service,Nbh_id,com_id,date_begin,date_end,Nbh_row,ADI,CV2,Forecastability
0,2328,2112 W Peterson Ave,-87.683593,41.991181,11.0,2.4,2.0,2016-01-01,NaN,6,5.393218,6.124525,Lumpy
1,2328,Clark St & Elmdale Ave,-87.669762,41.990683,13.0,2.4,2.0,2016-01-01,NaN,6,5.393218,6.124525,Lumpy
2,2328,351,-87.606611,41.803023,15.0,39.1,39.0,2016-01-01,NaN,114,3.326658,3.843092,Lumpy
3,2328,Cottage Grove Ave & 43rd St,-87.606587,41.816495,11.0,39.1,39.0,2016-01-01,NaN,114,3.326658,3.843092,Lumpy
4,2328,Cottage Grove Ave & 47th St,-87.606764,41.809857,13.0,39.1,39.0,2016-01-01,NaN,114,3.326658,3.843092,Lumpy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,713,Wentworth Ave & Cermak Rd,-87.631695,41.853077,17.0,3.4,3.0,2020-06-03,NaN,11,3.645452,4.719925,Lumpy
823,1597,Wentworth Ave & Cermak Rd (Temp),-87.631474,41.853059,19.0,3.4,3.0,2018-01-01,2020-06-04,11,3.645452,4.719925,Lumpy
824,2328,Wallace St & 35th St,-87.641292,41.830621,13.0,34.2,34.0,2016-01-01,NaN,101,6.902124,7.322397,Lumpy
825,588,Western Ave & 104th St,-87.662560,41.704878,15.0,72.2,72.0,2020-10-06,NaN,158,inf,NaN,Lumpy


In [68]:
# create empty map zoomed in on San Francisco
map2 = folium.Map(location=CHICAGO_COORDINATES, zoom_start=11)
####
colormap2 = cmp.StepColormap(
 eval(sns.color_palette('rocket').as_hex().__str__()),
 vmin=df_station['station_age'].min(), vmax=df_station['station_age'].max(),
  #for change in the colors, not used fr linear
 caption='Color Scale for Map'    #Caption for Color scale or Legend
)
colormap2


In [69]:
# add a marker for every record in the filtered data
for idx,row in df_station.iterrows():
    folium.Circle(
        location = (row['lat'],row['lng']),
        radius = 2,
        color = colormap2(row['station_age'])
        ).add_to(map2)

In [70]:
map2.add_child(colormap2,name='Age of station (days)')
display(map2)